### Prerequisite: Sentinel Hub account

In [1]:
INSTANCE_ID = 'f3a0f77b-301f-4517-889c-9fb040c0d7d1'

In [2]:
LAYER_NAME = 'TRUE-COLOR'  # e.g. TRUE-COLOR-S2-L1C

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# Import libraries
import datetime

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import json
from datetime import datetime

from sentinelhub import WmsRequest, BBox, CRS, MimeType, CustomUrlParam, get_area_dates
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest

c:\users\trung\appdata\local\continuum\anaconda3\envs\sentinel\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# Get coordinates of a tile from its geojson file
def get_coordinates(geojson_path):
    with open(geojson_path) as f:
        data = json.load(f)
        coordinates = data['features'][0]['geometry']['coordinates']
        x1 = coordinates[0][1][0]
        y1 = coordinates[0][1][1]
        x2 = coordinates[0][3][0]
        y2 = coordinates[0][3][1]
    return [x1, y1, x2, y2]

In [6]:
# Get bands of time-serires satellite images for an intended tile
def get_bands(geojson, start_date, end_date):
    """
    We use bands B02, B03, B04, B08, B11 and B12 
    for cloud/cloud shadow detection and compute NDVI/GNDVI
    """
    bbox_coords_wgs84 = get_coordinates(geojson)
    bounding_box = BBox(bbox_coords_wgs84, crs=CRS.WGS84)
    
    
    bands_script = 'return [B02,B03,B04,B08,B11,B12]'
    wms_bands_request = WmsRequest(layer=LAYER_NAME,
                               custom_url_params={
                                   CustomUrlParam.EVALSCRIPT: bands_script,
                                   CustomUrlParam.ATMFILTER: 'NONE'
                               },
                               bbox=bounding_box, 
                               time=(start_date, end_date), 
                               width=512, height=None,
                               image_format=MimeType.TIFF_d32f,
                               instance_id=INSTANCE_ID)
    
    dates = wms_bands_request.get_dates()
    wms_bands = wms_bands_request.get_data()
    
    return (wms_bands, dates)

In [7]:
# Get the tile name (pos) and its relative path
def get_pos_tile_path(geojson):
    pos = geojson.split('/')[4].split('.')[0][4:]
    tile_path = '../../data/phase-02/tile-' + pos
    return (pos, tile_path)

In [8]:
# Compute NDVI and GNDVI for all images
def compute_ndvi_gndvi(geojson, start_date, end_date):
    """
    Two-step processing:
    1. Use CSD-SI algorithm to detect and remove cloud/cloud shadow
    2. Compute NDVI/GNDVI from cloud-removed images
    """
    
    # Get bands
    print('Getting bands...')
    
    wms_bands, dates = get_bands(geojson, start_date, end_date)    
    size = len(dates)
    
    pos, tile_path = get_pos_tile_path(geojson)
    ndvi_path = tile_path + '/sen-ndvi/'
    gndvi_path = tile_path + '/sen-gndvi/'
    
    # Set parameters for CSD-SI algorithm
    T1=4/5 # 0.01, 0.1, 1, 10, 100 
    t2=1/2 # 1/10, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4, 1/3,1/2
    t3=1/6 # 1/4, 1/3, 1/2, 2/3, 3/4
    t4=1/4 # 1/2, 2/3, 3/4, 4/5, 5/6
    
    # For each date...
    for i in range(size):
        
        date = dates[i].strftime('%Y-%m-%d')
        print('---', date)
        
        print('Detecting cloud...')
        
        ## Get bands
        blue = wms_bands[i][:,:,0]
        green = wms_bands[i][:,:,1]
        red = wms_bands[i][:,:,2]
        nir = wms_bands[i][:,:,3]
        swir1 = wms_bands[i][:,:,4]
        swir2 = wms_bands[i][:,:,5]
        
        ## Compute CI1, CI2, CSI
        CI1 = (nir.astype(float) + 2 * swir1.astype(float))/(blue + green + red)
        CI2 = (blue.astype(float) + green.astype(float) + red.astype(float) + nir.astype(float) + swir1.astype(float) + swir2.astype(float))/6
        CSI = (nir.astype(float) + swir1.astype(float))/2

        ## Compute T2, T3, T4
        T2 = np.mean(CI2) + t2 * (np.max(CI2) - np.mean(CI2))
        T3 = np.min(CSI) + t3 * (np.mean(CSI) - np.min(CSI))
        T4 = np.min(blue) + t4 * (np.mean(blue) - np.min(blue))

        ## Cloud mask
        cloud_mask = np.zeros((512,512))
        for x in range(512):
            for y in range(512):
                if np.abs(CI1[x,y]-1) < T1:
                    cloud_mask[x,y]=1
                elif CI2[x,y] > T2:
                    cloud_mask[x,y]=1
                elif CSI[x,y] < T3 and blue[x,y] < T4:
                    cloud_mask[x,y]=1
                else:
                    cloud_mask[x,y]=0

        print('Computing NDVI, GNDVI...')
        
        ## Compute NDVI
        NDVI = (nir.astype(float)-red.astype(float))/(nir+red)
        GNDVI = (nir.astype(float)-green.astype(float))/(nir+green)
        
        for x in range(512):
            for y in range(512):
                if cloud_mask[x,y] == 1:
                    NDVI[x,y] = np.NaN
                    GNDVI[x,y] = np.NaN
                    
        print('Saving...')
        
        # Save NDVI
        ndvi_csv_path = ndvi_path + 'sen-ndvi-' + pos + '-' + date + '.csv'
        print(ndvi_csv_path)
        np.savetxt(ndvi_csv_path, NDVI, delimiter=",")

        # Save GNDVI
        gndvi_csv_path = gndvi_path + 'sen-gndvi-' + pos + '-' + date + '.csv'
        np.savetxt(gndvi_csv_path, GNDVI, delimiter=",")
        
        # Save NDVI plot
        ndvi_png_path = ndvi_path + 'sen-ndvi-' + pos + '-' + date + '.png'
        plt.imshow(NDVI)
        plt.clim(0, 1)
        plt.colorbar()
        plt.axis('off')
        plt.savefig(ndvi_png_path, bbox_inches='tight', pad_inches=0, dpi=169.6)
        plt.close()
        
        # Save GNDVI plot
        gndvi_png_path = gndvi_path + 'sen-gndvi-' + pos + '-' + date + '.png'
        plt.imshow(GNDVI)
        plt.clim(0, 1)
        plt.colorbar()
        plt.axis('off')
        plt.savefig(gndvi_png_path, bbox_inches='tight', pad_inches=0, dpi=169.6)
        plt.close()
        
        print('Done!')

In [9]:
# Choose dates
start_date = '2015-11-18'
end_date = '2019-10-29'

# Choose the tile
geojson = '../../data/geometries/geo-x6144-y5632.geojson'
print(geojson)

# Compute and save all NDVI/GNDVI values and plots 
compute_ndvi_gndvi(geojson, start_date, end_date)

../../data/geometries/geo-x6144-y5632.geojson
Getting bands...
--- 2015-11-18
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2015-11-18.csv
Done!
--- 2015-11-28
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2015-11-28.csv
Done!
--- 2015-11-28
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2015-11-28.csv
Done!
--- 2016-02-06
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2016-02-06.csv
Done!
--- 2016-03-07
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2016-03-07.csv
Done!
--- 2016-03-27
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2016-03-27.csv
D

Done!
--- 2017-07-20
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-07-20.csv
Done!
--- 2017-07-20
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-07-20.csv
Done!
--- 2017-07-25
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-07-25.csv
Done!
--- 2017-07-30
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-07-30.csv
Done!
--- 2017-08-04
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-08-04.csv
Done!
--- 2017-08-09
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2017-08-09.csv
Done!
--- 2017-08-14
Detecting cloud...
Computing NDVI, GN

Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-04-11.csv
Done!
--- 2018-04-16
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-04-16.csv
Done!
--- 2018-04-21
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-04-21.csv
Done!
--- 2018-04-26
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-04-26.csv
Done!
--- 2018-05-06
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-05-06.csv
Done!
--- 2018-05-11
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-05-11.csv
Done!
--- 2018-05-16
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/til

Done!
--- 2018-12-27
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2018-12-27.csv
Done!
--- 2019-01-01
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-01-01.csv
Done!
--- 2019-01-06
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-01-06.csv
Done!
--- 2019-01-11
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-01-11.csv
Done!
--- 2019-01-16
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-01-16.csv
Done!
--- 2019-01-21
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-01-21.csv
Done!
--- 2019-01-26
Detecting cloud...
Computing NDVI, GN

Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-09-23.csv
Done!
--- 2019-09-28
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-09-28.csv
Done!
--- 2019-10-03
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-10-03.csv
Done!
--- 2019-10-08
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-10-08.csv
Done!
--- 2019-10-13
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-10-13.csv
Done!
--- 2019-10-18
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/tile-x6144-y5632/sen-ndvi/sen-ndvi-x6144-y5632-2019-10-18.csv
Done!
--- 2019-10-23
Detecting cloud...
Computing NDVI, GNDVI...
Saving...
../../data/phase-02/til